In [1]:
!pip install sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 3.4 MB/s eta 0:00:00a 0:00:01


Load dataset


In [ ]:
from datasets import load_dataset

datafiles = {'test': 'embed_data/test_embed.jsonl', 'train': 'embed_data/train_embed.jsonl'}

dataset = load_dataset('json', data_files = datafiles, split = {'train': 'train', 'test': 'test'})

train_test_ds = dataset['train'].train_test_split(test_size=0.1)

train_ds = train_test_ds['train']
val_ds = train_test_ds['test']
test_ds = dataset['test']

def transfrom(example):
    example['pos'] = example['pos'][0]
    example['neg'] = example['neg'][0]
    return example

train_ds = train_ds.map(transfrom)
val_ds = val_ds.map(transfrom)
test_ds = test_ds.map(transfrom)

print(val_ds)

Generating test split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/32785 [00:00<?, ? examples/s]

Map:   0%|          | 0/3643 [00:00<?, ? examples/s]

Map:   0%|          | 0/8135 [00:00<?, ? examples/s]

Dataset({
    features: ['query', 'pos', 'neg'],
    num_rows: 3643
})


Load mô hình


In [ ]:
from sentence_transformers import SentenceTransformer
import torch
model = SentenceTransformer("BAAI/bge-m3", model_kwargs = {'torch_dtype': torch.float16})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Đánh giá mô hình gốc


In [ ]:
from sentence_transformers.evaluation import TripletEvaluator, SimilarityFunction

evaluator = TripletEvaluator(
    anchors=val_ds["query"],
    positives=val_ds["pos"],
    negatives=val_ds["neg"],
    main_distance_function=SimilarityFunction.COSINE,
)

In [11]:
evaluator(model)

{'cosine_accuracy': 0.9615701345045292,
 'dot_accuracy': 0.0381553664562174,
 'manhattan_accuracy': 0.959923140269009,
 'euclidean_accuracy': 0.9615701345045292,
 'max_accuracy': 0.9615701345045292}

Lưu vào local


In [ ]:
model.save_pretrained("ebmodel/bge-m3-ft-triplet")